In [1]:
# setting up the libraries that we will need 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV, cross_val_score
#from mlxtend.preprocessing import minmax_scaling
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [7]:
# loading the data
train = pd.read_csv("train_data.csv")
test = pd.read_csv("test_data.csv")
train.name = "Train"
test.name = "Test"
train

# train.drop(['time', 'lat', 'sid', 'lon', 'lid', 't_user_id', 'altitude'], axis = 1)
# test.drop(['time', 'lat', 'lon', 'lid', 't_user_id', 'altitude'], axis = 1)

array([[0, 'bike', 21.22594101, 2.653242626],
       [1, 'bike', 15.90202918, 1.135859227],
       [2, 'bike', 20.31214179, 1.128452322],
       ...,
       [969, 'walk', 43.19932676, 1.4399775590000001],
       [970, 'walk', 627.421457, 0.7745943909999999],
       [971, 'walk', 0.0, 0.0]], dtype=object)

In [ ]:
# def corr(df):
#     corr_matrix = df.corr()
#     print(corr_matrix["label"])

In [ ]:
#data exploration
train.head()

In [117]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972 entries, 0 to 971
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  972 non-null    int64  
 1   label       972 non-null    object 
 2   distance    972 non-null    float64
 3   speed       972 non-null    float64
 4   acc         972 non-null    float64
 5   bearing     972 non-null    float64
 6   jerk        972 non-null    float64
 7   brate       972 non-null    float64
 8   brrate      972 non-null    float64
dtypes: float64(7), int64(1), object(1)
memory usage: 68.5+ KB


In [118]:
train.describe()


,Unnamed: 0,distance,speed,acc,bearing,jerk,brate,brrate
count,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000
mean,485.500000,2074.849504,12.339225,-0.049772,172.893953,0.026876,-0.367543,0.266131
std,280.736531,9038.926789,25.716790,2.088298,82.226553,0.447048,9.841408,4.784026
min,0.000000,0.000000,0.000000,-17.755512,0.000000,-5.815833,-157.864232,-34.853519
25%,242.750000,24.745137,0.796439,-0.028523,119.883024,-0.001020,-0.375743,-0.017068
50%,485.500000,117.679985,3.679879,0.000050,176.157678,0.000005,-0.000579,-0.000007
75%,728.250000,963.552528,24.873571,0.013408,215.059660,0.001235,0.412344,0.018026
max,971.000000,137847.550100,698.878256,57.088406,359.687272,8.401501,69.197754,84.147085


In [119]:
# corr(train)


In [120]:
test.describe()


,Unnamed: 0,distance,speed,acc,bearing,jerk,brate,brrate
count,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000
mean,485.500000,2074.849504,12.339225,-0.049772,172.893953,0.026876,-0.367543,0.266131
std,280.736531,9038.926789,25.716790,2.088298,82.226553,0.447048,9.841408,4.784026
min,0.000000,0.000000,0.000000,-17.755512,0.000000,-5.815833,-157.864232,-34.853519
25%,242.750000,24.745137,0.796439,-0.028523,119.883024,-0.001020,-0.375743,-0.017068
50%,485.500000,117.679985,3.679879,0.000050,176.157678,0.000005,-0.000579,-0.000007
75%,728.250000,963.552528,24.873571,0.013408,215.059660,0.001235,0.412344,0.018026
max,971.000000,137847.550100,698.878256,57.088406,359.687272,8.401501,69.197754,84.147085


In [121]:
coordinates_columns = ['distance', 'speed', 'acc', 'bearing', 'jerk', 'brate', 'brrate']
data_set = [train, test]
for coord in coordinates_columns:
    for data in data_set:
        maxi = data[coord].max()
        mini = data[coord].min()
        print ("Range of {} in {} data is : ({:.3f}, {:.3f})".format(coord, data.name, maxi, mini))
del data_set

Range of distance in Train data is : (137847.550, 0.000)
Range of distance in Test data is : (137847.550, 0.000)
Range of speed in Train data is : (698.878, 0.000)
Range of speed in Test data is : (698.878, 0.000)
Range of acc in Train data is : (57.088, -17.756)
Range of acc in Test data is : (57.088, -17.756)
Range of bearing in Train data is : (359.687, 0.000)
Range of bearing in Test data is : (359.687, 0.000)
Range of jerk in Train data is : (8.402, -5.816)
Range of jerk in Test data is : (8.402, -5.816)
Range of brate in Train data is : (69.198, -157.864)
Range of brate in Test data is : (69.198, -157.864)
Range of brrate in Train data is : (84.147, -34.854)
Range of brrate in Test data is : (84.147, -34.854)


In [122]:
X_train= train
labels = X_train["label"].copy()
X_train = X_train.drop("label", axis=1)
X_test = test
 
del train, test

In [123]:
X_train.head()

,Unnamed: 0,distance,speed,acc,bearing,jerk,brate,brrate
0,0,21.225941,2.653243,-0.189673,290.447082,0.023643,-19.508054,4.003895
1,1,15.902029,1.135859,-0.000529,134.382649,0.019251,12.523108,-1.397089
2,2,20.312142,1.128452,0.268987,309.706160,-0.016452,-7.036139,0.446452
3,3,746.277432,5.970219,-0.027141,183.055660,-0.000987,1.000006,0.002074
4,4,18.043469,2.577638,-0.150516,308.056352,0.037949,1.259286,-1.943406


In [124]:
param_grid = [
    {'n_estimators': [3, 10, 30, 40, 50], 'max_features': [2, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
]

In [125]:
#loading model
forest_reg = RandomForestRegressor()
# grid search to determine the best set of HP using grid search and 5 folders for cross validation
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring = 'neg_mean_squared_error',
                          return_train_score=True)

In [126]:
#fitting and selecting best set of parameters
grid_search.fit(X_train, labels)
best_model = grid_search.best_estimator_
grid_search.best_params_

ValueError: could not convert string to float: 'bike'